## 1. Google Drive 마운트 (필수)
모델 파일(약 5GB 이상)을 매번 다운로드하지 않기 위해 Google Drive를 연결합니다.
또한 프로젝트 파일(`law_project.zip`)을 불러오는 용도로도 사용됩니다.

In [ ]:
from google.colab import drive
import os

if not os.path.exists("/content/drive"):
    drive.mount("/content/drive")


## 2. 환경 설정 및 Ollama 설치
Linux용 Ollama 바이너리와 필요한 Python 패키지를 설치합니다.
**GPU 감지 도구(pciutils) 포함**

In [ ]:
# 1. Install System Dependencies (Fix missing zstd, pciutils)
!sudo apt-get update && sudo apt-get install -y zstd pciutils lshw
# Check GPU status
!nvidia-smi

# 2. Install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# 3. Install Python Dependencies for Web & Tunneling
!pip install pyngrok uvicorn nest_asyncio

In [ ]:
# 4. Start Ollama Server in Background (Local Execution)
import subprocess
import time
import os
import requests

# [안정성 수정] Google Drive 직접 실행 대신 로컬 실행으로 변경 (충돌 방지)
DRIVE_MODEL_PATH = "/content/drive/MyDrive/OllamaModels"
LOCAL_MODEL_PATH = "/usr/share/ollama/.ollama/models" # Default path or ~/.ollama

# Ollama server start (Clean environment)
if "OLLAMA_MODELS" in os.environ:
    del os.environ["OLLAMA_MODELS"]

print("Starting Ollama server...")
process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Wait for Ollama to become ready with Timeout
max_retries = 20
print("Waiting for Ollama to start...")
for i in range(max_retries):
    try:
        response = requests.get("http://localhost:11434")
        if response.status_code == 200:
            print("Ollama server is running successfully!")
            break
    except requests.exceptions.ConnectionError:
        if i == max_retries - 1:
            print("Failed to connect to Ollama server.")
            # Print potential errors
            # Note: process.communicate() blocks, so we just peek if it's dead
            if process.poll() is not None:
                out, err = process.communicate()
                print(f"Process terminated. Out: {out}, Err: {err}")
        time.sleep(2)


In [ ]:
!ollama pull qwen2.5
!ollama pull nomic-embed-text

## 4. 프로젝트 파일 준비
Google Drive에서 `law_project.zip`을 가져와 실행 환경을 구축합니다.

In [ ]:
import shutil

DRIVE_PROJECT_PATH = "/content/drive/MyDrive/LawProject/law_project.zip"

# 1. 이미 파일이 있으면 (재실행 시) 패스
if os.path.exists("law_project.zip"):
    print("law_project.zip already exists locally.")
else:
    if os.path.exists(DRIVE_PROJECT_PATH):
        print(f"Google Drive에서 파일을 복사합니다: {DRIVE_PROJECT_PATH}")
        shutil.copy(DRIVE_PROJECT_PATH, "/content/law_project.zip")
    else:
        print("!!! Google Drive에 파일이 없습니다. 직접 업로드를 시도합니다. !!!")
        from google.colab import files
        uploaded = files.upload()

In [ ]:
# Unzip with CP949 (Korean) encoding
!unzip -O cp949 -o law_project.zip -d /content/LawProject

In [ ]:
%cd /content/LawProject
!pip install -r requirements.txt

## 7. 웹 서버 실행 (Ngrok)
웹 인터페이스를 외부에서 접속할 수 있도록 `ngrok`을 설정합니다.
**[Ngrok Dashboard](https://dashboard.ngrok.com/get-started/your-authtoken)**에서 토큰을 복사해오세요.

In [ ]:
from pyngrok import ngrok
import getpass

print("Ngrok Authtoken을 입력하세요 (입력 시 보이지 않습니다):")
token = getpass.getpass()
ngrok.set_auth_token(token)

# Terminate open tunnels if any
ngrok.kill()

# Open an HTTP tunnel on port 8000
public_url = ngrok.connect(8000)
print(f"\n>>> 웹 인터페이스 접속 주소: {public_url} <<<\n")

In [ ]:
# Run FastAPI Server via Uvicorn
import uvicorn
import nest_asyncio
from app.server import app

# Apply nest_asyncio to allow nested event loops in Jupyter
nest_asyncio.apply()

print("서버를 시작합니다... 위에서 출력된 ngrok URL로 접속하세요.")
uvicorn.run(app, host="0.0.0.0", port=8000)